![LoL](https://i.blogs.es/19ba21/league-of-legends/1366_2000.jpg)

Introducción:

Al ser un jugador constante de League of Legends, pensé que sería interesante obtener los datos de las últimas partidas de cualquier jugador del mundo. Es por eso que decidí usar la API de Riot Games, la empresa encargada del desarrollo del juego.

El problema:

A lo largo de la obtención de datos me encontré con diferentes problemas como que la información del tipo de partida, el campeón o personaje usado y la temporada en la que se realizaba la partida estaban identificadas con su ID, lo cual no nos servía para el análisis, ya que no había forma de saber que representaba ese ID.

Solución:

Al encontrar este problema realicé dos cosas:

1. Realicé un scraping de una página de riot para obtener como resultado dos tablas, la primera de ellas con el ID de la temporada y el nombre de dicha temporada, y la segunda con el ID del tipo de partida y su descripción.

2. Investigando un poco encontré otra api llamada data dragon, con la cuál pude obtener algunos campeones y sus ID.

Descripción:

Fuente: https://api.riotgames.com/ | http://ddragon.leagueoflegends.com/cdn/6.24.1/data/en_US/champion.json
Tipo de fuente: API
Autor del estudio: Diego Quiroz
Lenguaje programación utilizado: Python
Librerías utilizadas: Requests, Json, Pandas, datetime.

Objetivos del estudio:

Obtener un csv con las últimas 100 partidas del jugador seleccionado.

Áreas de oportunidad:

1. Encontrar una base completa con los ID de cada campeón.
2. Realizar una selección de las mejores 10 partidas.
3. Relacionar dichas 10 partidas con otra conexión api, para ver el desarrollo de la partida.

In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
regiones = {'BR':'BR1',
'EUNE':'EUN1',
'EUW':'EUW1',
'JP':'JP1',
'KR':'KR',
'LAN':'LA1',
'LAS':'LA2',
'NA':'NA1',
'OCE':'OC1',
'TR':'TR1',
'RU':'RU',
'PBE':'PBE1'
}

In [3]:
summoner = input('¿Qué summoner quieres buscar?  ')
input_region = input('¿Cuál es su región? eligé entre las siguientes opciones BR, EUNE, EUW, JP, KR, LAN, LAS, NA, OCE, TR, RU, PBE  ')
region = regiones[input_region]
url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}?api_key=RGAPI-55eeb8cb-cfac-4ba8-bb06-ef7690fad7bc'
LOL = requests.get (url)
LOL_json = LOL.json()

¿Qué summoner quieres buscar?  The Wardman
¿Cuál es su región? eligé entre las siguientes opciones BR, EUNE, EUW, JP, KR, LAN, LAS, NA, OCE, TR, RU, PBE  NA


In [4]:
summoner_Id = LOL_json['id']
accountId = LOL_json['accountId']
puuid = LOL_json['puuid']

In [5]:
url2 = f'https://{region}.api.riotgames.com//lol/match/v4/matchlists/by-account/{accountId}?api_key=RGAPI-55eeb8cb-cfac-4ba8-bb06-ef7690fad7bc'
matches = requests.get (url2)
matches_json = matches.json()


In [6]:
matches_df = pd.DataFrame(matches_json['matches'])
matches_df.to_csv('matches API.csv', index = False)

In [7]:
churl = 'http://ddragon.leagueoflegends.com/cdn/6.24.1/data/en_US/champion.json'
champions =requests.get(churl)
champions_json = champions.json()
champions_info = champions_json['data']

In [8]:
champions_df = pd.DataFrame(champions_info)
champions_df = champions_df.transpose()
champions_dict = dict(zip(dict(champions_df['key']).values(),dict(champions_df['key']).keys()))

In [9]:
matches_df['Summoner'] = summoner
matches_df['role'] = matches_df['role'].str.split('_', expand = True)
matches_df['role'] = matches_df['role']
matches_df['Fecha y Hora'] = pd.to_datetime(matches_df['timestamp'],unit='ms')
matches_df['Champion Name'] = [champions_dict[str(a)] for a in list(matches_df['champion'])]

In [10]:
season = pd.read_csv('season final.csv')
queue = pd.read_csv('queue final.csv')

In [11]:
matches_df= pd.merge(matches_df,queue, on='queue')
matches_df= pd.merge(matches_df,season, on='season')

In [12]:
matches_df.drop(['champion','gameId','queue','season','timestamp'], axis = 1, inplace=True)

In [13]:
cols_name = ['Lane','Region','Solo/Duo','Summoner','Fecha y Hora','Champion','Map','Type','Season']
matches_df = matches_df.rename(columns = dict(zip(matches_df.columns,cols_name)))

In [14]:
cols_order = ['Summoner','Region','Fecha y Hora','Solo/Duo','Type','Map','Lane','Champion','Season']
matches_df = matches_df[cols_order]
matches_df.to_csv(f"{summoner}'s last 100 matches.csv", index=False)

In [15]:
matches_df

,Summoner,Region,Fecha y Hora,Solo/Duo,Type,Map,Lane,Champion,Season
0,The Wardman,NA1,2019-07-18 18:47:57.269,DUO,5v5 ARAM games,Howling Abyss,NONE,Graves,SEASON 2019
1,The Wardman,NA1,2019-07-02 00:59:14.090,DUO,5v5 Ranked Solo games,Summoner's Rift,TOP,Mordekaiser,SEASON 2019
2,The Wardman,NA1,2019-06-27 03:23:27.175,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Thresh,SEASON 2019
3,The Wardman,NA1,2019-06-26 06:59:58.005,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Twitch,SEASON 2019
4,The Wardman,NA1,2019-06-26 06:13:37.753,SOLO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Sivir,SEASON 2019
5,The Wardman,NA1,2019-06-26 05:40:55.337,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Jinx,SEASON 2019
6,The Wardman,NA1,2019-06-26 05:02:09.996,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Jinx,SEASON 2019
7,The Wardman,NA1,2019-06-26 02:44:46.587,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Lux,SEASON 2019
8,The Wardman,NA1,2019-06-26 01:56:25.799,DUO,5v5 Ranked Solo games,Summoner's Rift,MID,Morgana,SEASON 2019
9,The Wardman,NA1,2019-06-26 01:20:22.169,DUO,5v5 Ranked Solo games,Summoner's Rift,BOTTOM,Nautilus,SEASON 2019
